# 2. Score classic methods

In [1]:
import ast
import pandas as pd

from common import score_models, clean_string, load_preds

In [2]:
reading_lists = pd.read_csv("../reading_lists.csv")
reading_lists['reading_list'] = reading_lists['reading_list'].apply(ast.literal_eval)

trues = { id_:[clean_string(ref["title"]) for ref in references if ref["acl_id"]] for id_, references in zip(reading_lists["id"], reading_lists["reading_list"]) }

In [3]:
models_paths = {
    "Semantic Scholar (Any)": "classic_methods/preds/semantic_scholar_any",
    "Semantic Scholar (ACL)": "classic_methods/preds/semantic_scholar_acl",
    "BM25": "classic_methods/preds/bm25",
    "SPECTER2": "classic_methods/preds/specterv2",
}

## Default comparison

In [8]:
table = score_models(trues, models_paths.keys(), models_paths.values(), split_by_years=False, use_title_instead_of_id=True); table 
#print(table.to_latex(float_format="{:0.1f}".format))

recall                                     ndcg  \
                              A1        A2         A3      Mean        A1   
Semantic Scholar (Any)  4.249837  2.720157   6.673190  4.547728  2.130474   
Semantic Scholar (ACL)  8.421396  5.225049  10.202218  7.949554  5.526876   
BM25                    8.873668  9.174821  11.156773  9.735087  5.722704   
SPECTER2                5.787671  5.676778   8.245864  6.570104  3.064759   

                                                           mrr            \
                              A2        A3      Mean        A1        A2   
Semantic Scholar (Any)  1.506814  3.930081  2.522456  2.463178  1.955588   
Semantic Scholar (ACL)  3.714255  7.488272  5.576468  7.633369  5.254403   
BM25                    5.467598  6.581614  5.923972  8.765831  6.271754   
SPECTER2                3.397064  4.709815  3.723880  3.333865  4.578341   

                                             
                               A3      Mean  
Semantic Scholar (Any)   5.617313  3.345360  
Semantic Scholar (ACL)  11.205697  8.031156  
BM25                     9.199094  8.078893  
SPECTER2                 5.763281  4.558496

## Matches of correct prediction

In [5]:
def match(trues, preds1, preds2):
    match = 0
    size = 0
    for k, trues_v in trues.items():
        if trues_v != []:
            match1 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds1[k]))
            match2 = set(dict.fromkeys(trues_v)) & set(dict.fromkeys(preds2[k]))
            match += len(match1 & match2)
            size += 1
    return match/size

In [6]:
bm25_preds = load_preds("classic_methods/preds/bm25", use_title_instead_of_id=True)
specter2_preds = load_preds("classic_methods/preds/specterv2", use_title_instead_of_id=True)

(match(trues, bm25_preds["A1"], specter2_preds["A1"]) +\
match(trues, bm25_preds["A2"], specter2_preds["A2"]) +\
match(trues, bm25_preds["A3"], specter2_preds["A3"])) /3

0.1050228310502283